# EXTRA


In [1]:
# Como no hemos encontrado ningún tweet que le llamen gordo al amado lider incluyo este extra de como sería un análisis de sentimiento
# de mensaje en twitter de manera muy muy básica

In [2]:
# De nuevo empezamos con los imports

In [3]:
import pandas as pd
import numpy as np
import csv
import json
import nltk
import re
import sys
import pickle
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from pprint import pprint
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
from nltk.classify import ClassifierI
from nltk.tokenize import word_tokenize
from nltk import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag.hmm import HiddenMarkovModelTagger
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from statistics import mode

print (sys.version)


2.7.12 |Anaconda 4.0.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


In [114]:
consumer_key = 'TOP SECRET'
consumer_secret = '+TOP SECRET'
access_token = '++TOP SECRET'
access_token_secret = 'requete TOP SECRET'

In [115]:
# Vamos a extrer los tweets

def search(hashtag, Max_Tweets, Recdata):

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    data = Cursor(api.search, q=hashtag, lang='es').items(Max_Tweets)
    
    # en carpeta data vamos a almacenar la información en un json.
    
    current_working_dir = "./data/"
    log_tweets = current_working_dir  + 'tweets.json'
    with open(log_tweets, 'w') as outfile:
        for tweet in data:
            Recdata.append(json.loads(json.dumps(tweet._json)))
            outfile.write(json.dumps(tweet._json))
            outfile.write("\n")
            
    # PCreamos un dataset con los tweets
    tweets = pd.DataFrame()
    tweets['text'] = map(lambda tweet: tweet['text'].encode('utf-8'), Recdata)
   
    # Creamos fichero csv
    tweets.to_csv('tweets.csv', sep=';', header=False)

In [116]:
# Vamos a ver los ultimos 1000 tweetss del hastaag coreadelnorte

Max_Tweets = 1000
tweets_data = []
hashtag = "#CoreaDelNorte"

search(hashtag, Max_Tweets, tweets_data)

# con esto nos hemos bajado los tweets en español del hastag corea del norte

In [4]:
# hacemos una lista con los tweets que hemos bajado

tweet_list = []
with open('./data/tweets.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tweet_list.append(tweet['text'].encode("utf-8"))

In [6]:
def read_tweets(fname, t_type):
    tweets = []
    f = open(fname, 'r')
    line = f.readline()
    while line != '':
        tweets.append([line, t_type])
        line = f.readline()
    f.close()
    return tweets

In [7]:
# Importamos corpus de palabras positivas y negativas (algo muy muy simple)

positivos = read_tweets('./positivo.csv', 'Positivo')
negativos = read_tweets('./negativo.csv', 'Negativo')

In [13]:
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [14]:
# hacemos una lista vacia

tweets = []

# Ahora juntammos las dos listas de palabras. 
# Despues lo ponemos en minusculas.
# en la lista tenemos palabra y etiqueta.

for (words, sentiment) in positivos + negativos:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    tweets.append((words_filtered, sentiment))

word_features = get_word_features(get_words_in_tweets(tweets))

# ponemos el training set y entrenamos el clasificador de Naive Bayes
training_set = nltk.classify.util.apply_features(extract_features, tweets)
classifier = NaiveBayesClassifier.train(training_set)

In [18]:
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector

In [27]:
#Proceso de Tweets
def processTweet(tweet):
    tweet = tweet.lower()
    # www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #@username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

stopWords = []

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)


In [28]:
# Primero entrenamos al ordenador en bbase a la lista de positivass y negativas que le hemos dado

NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

# aplicamos lo aprenndido a la lista de tweets y vemos como nos lo clasifca

for tweet in tweet_list:
        processedTestTweet = processTweet(tweet)
        print tweet + ' esto es  ' + NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet))) + '\n'

RT @amnistiaespana: Los campos penitenciarios de #CoreadelNorte, en perfecto estado de funcionamiento https://t.co/zxwIyY6Fp6 esto es  Positivo

RT @amnistiaespana: Los campos penitenciarios de #CoreadelNorte, en perfecto estado de funcionamiento https://t.co/zxwIyY6Fp6 esto es  Positivo

RT @SputnikMundo: Diez hechos sobre #CoreaDelNorte que no conocías y te dejarán estupefacto https://t.co/LLYmyxngFM https://t.co/aHzYZSpcF5 esto es  Negativo

https://t.co/TqXun1bFXO
La Otra Realidad de #CoreadelNorte 
#kimjongun #FelizJueves esto es  Negativo

RT @amnistiaespana: Los campos penitenciarios de #CoreadelNorte, en perfecto estado de funcionamiento https://t.co/zxwIyY6Fp6 esto es  Positivo

RT @SputnikMundo: Diez hechos sobre #CoreaDelNorte que no conocías y te dejarán estupefacto https://t.co/LLYmyxngFM https://t.co/aHzYZSpcF5 esto es  Negativo

RT @amnistiaespana: Los campos penitenciarios de #CoreadelNorte, en perfecto estado de funcionamiento https://t.co/zxwIyY6Fp6 esto es  Positivo


Como vemos ya nos esta clasificando los tweets como "esto es positivo" y "esto es negativo".
No obstante si lo miramos con atención como ha clasificado vemos que:

dice que "Los campos penitenciarios de corea del norte estan en perfecto funcionamiento"  es positivo.  Claro, eso es porque en nuestro listado de expresiones positivas esta perfecto. Pero tratandose de Corea del Norte es para pensarselo muy mucho si es realmente positivo.

Entonces estamos llegando al punto clave de NLP y es que hace falta no solo lingüistas sino más información para poder dar un correcto análisis de texto y sentimiento de un texto

Gracias

<img src="analisis.jpg"> 